# Argumentation Structurée

**Navigation**: [← Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7-Advanced-Argumentation →](Tweety-7-Advanced-Argumentation.ipynb)

---

## Objectifs pédagogiques

1. Maîtriser le framework ASPIC+ et sa conversion vers Dung
2. Découvrir DeLP (Defeasible Logic Programming)
3. Explorer ABA (Assumption-Based Argumentation)
4. Comprendre l'argumentation déductive en logique propositionnelle
5. Utiliser ASP (Answer Set Programming) avec Clingo

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Note**: ASP nécessite Clingo installé. Le notebook configure automatiquement le chemin si Clingo est disponible.

In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

### 4.2 ASPIC+
<a id="4.2"></a>

ASPIC+ est un framework mature et largement utilisé pour l'argumentation **structurée**. Contrairement à Dung où les arguments sont abstraits, ASPIC+ construit des arguments logiques à partir d'une base de connaissances et de règles d'inférence, en distinguant règles strictes et règles défaisables.

* **Base de Connaissances (KB)**: Contient des axiomes (faits certains) et des règles.
* **Règles d'inférence**:
    * Strictes (`StrictInferenceRule`, `->`): Si les prémisses sont acceptées, la conclusion l'est nécessairement.
    * Défaisables (`DefeasibleInferenceRule`, `=>`): Si les prémisses sont acceptées, la conclusion l'est plausiblement, mais la règle elle-même peut être attaquée (undercutting) ou la conclusion réfutée (rebutting).
* **Préférences**: Un ordre (souvent partiel) peut être défini sur les règles défaisables pour résoudre les attaques "rebutting".
* **Arguments**: Construits par chaînage de règles depuis les axiomes.
* **Attaques**: Peuvent viser la conclusion (rebutting), une sous-conclusion (undermining sur conclusion intermédiaire) ou une règle défaisable (undercutting).
* **`AspicArgumentationTheory`**: Représente la théorie ASPIC+. Nécessite un `RuleFormulaGenerator` pour spécifier la logique sous-jacente (ex: `PlFormulaGenerator` pour la logique propositionnelle).
* **Conversion vers Dung (`asDungTheory`)**: Permet d'analyser la théorie ASPIC+ en la transformant en un AAF standard, sur lequel on peut appliquer les sémantiques de Dung (Grounded, Preferred, etc.).

L'exemple suivant utilise la logique propositionnelle comme langage sous-jacent, basé sur `AspicExample.java`.

In [ ]:
# --- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---
print("\n--- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ASPIC+...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        # Logique Propositionnelle comme base
        from org.tweetyproject.logics.pl.syntax import Proposition, Negation, PlFormula
        # Classes ASPIC+
        from org.tweetyproject.arg.aspic.syntax import AspicArgumentationTheory, DefeasibleInferenceRule, StrictInferenceRule
        from org.tweetyproject.arg.aspic.ruleformulagenerator import PlFormulaGenerator # Crucial pour PL
        # Classes Dung pour la conversion et le raisonnement
        from org.tweetyproject.arg.dung.syntax import Argument as DungArgument, Attack as DungAttack, DungTheory
        from org.tweetyproject.arg.dung.reasoner import SimpleGroundedReasoner

        print("✔️ Imports ASPIC+ (PL) et Dung réussis.")

        # --- Théorie ASPIC+ ---
        # Nécessite un générateur pour la logique sous-jacente (ici PL)
        pl_formula_generator = PlFormulaGenerator()
        aspic_theory = AspicArgumentationTheory(pl_formula_generator)
        # Spécifier aussi le générateur pour les formules DANS les règles
        aspic_theory.setRuleFormulaGenerator(pl_formula_generator)

        # Propositions
        a = Proposition("a"); b = Proposition("b"); c = Proposition("c"); d_prop = Proposition("d") # Renommé

        # Règles Défaisables (=>)
        # r1: b, c => a
        r1_def = DefeasibleInferenceRule()
        r1_def.setConclusion(a)
        r1_def.addPremise(b); r1_def.addPremise(c)
        aspic_theory.addRule(r1_def)

        # r2: b => d
        r2_def = DefeasibleInferenceRule()
        r2_def.setConclusion(d_prop)
        r2_def.addPremise(b)
        aspic_theory.addRule(r2_def)

        # r3: a => !d
        r3_def = DefeasibleInferenceRule()
        r3_def.setConclusion(Negation(d_prop))
        r3_def.addPremise(a)
        aspic_theory.addRule(r3_def)

        # (Pas de règles strictes dans cet exemple)

        # Axiomes (faits certains ->)
        aspic_theory.addAxiom(b) # -> b
        aspic_theory.addAxiom(c) # -> c

        print("\n--- Théorie ASPIC+ ---")
        print(str(aspic_theory)) # Utilise toString()

        # --- Conversion en Cadre de Dung ---
        print("\n--- Conversion en AF de Dung ---")
        try:
            # asDungTheory() génère le graphe d'attaque basé sur la théorie ASPIC+
            dung_equivalent = aspic_theory.asDungTheory()

            print("Arguments générés par ASPIC+:")
            args_aspic = dung_equivalent.getNodes()
            if args_aspic.isEmpty():
                print("  (Aucun argument généré)")
            else:
                for arg in args_aspic:
                    # L'affichage d'un argument ASPIC+ peut être complexe
                    print(f"  - {arg}")

            print("\nAttaques générées par ASPIC+:")
            attacks_aspic = dung_equivalent.getAttacks()
            if attacks_aspic.isEmpty():
                 print("  (Aucune attaque générée)")
            else:
                for att in attacks_aspic:
                    print(f"  - {att}")

            # --- Raisonnement sur l'AF équivalent ---
            print("\nRaisonnement sur l'AF de Dung équivalent:")
            grounded_reasoner_dung = SimpleGroundedReasoner()
            grounded_extension = grounded_reasoner_dung.getModel(dung_equivalent)
            print(f" - Extension Grounded : {grounded_extension}")

            # Pourrait-on vérifier si 'a' est acceptable ?
            # Il faudrait trouver l'argument ASPIC correspondant à 'a'
            # C'est plus complexe, on se contente de l'extension pour l'instant.

        except jpype.JException as e_conv_java:
            print(f"❌ Erreur Java lors de la conversion ASPIC->Dung: {e_conv_java.message()}")
            # print(e_conv_java.stacktrace())
        except Exception as e_conv_py:
            print(f"❌ Erreur Python lors de la conversion ASPIC->Dung: {e_conv_py}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour ASPIC+ : {e}. Vérifiez le JAR 'arg.aspic'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple ASPIC+: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple ASPIC+: {e_gen}")
        import traceback
        traceback.print_exc()

### 4.3 Defeasible Logic Programming (DeLP)
<a id="4.3"></a>

*(Section à compléter avec l'exemple `DeLPExample.java`)*

DeLP combine la programmation logique avec le raisonnement défaisable. Il utilise des règles strictes et des règles défaisables (`-<`). Un argument est construit pour supporter un littéral, et la notion de "warrant" (justification) est déterminée en comparant les arguments pour et contre ce littéral, en utilisant un critère de comparaison comme la spécificité généralisée.

In [ ]:
# --- 4.3 Defeasible Logic Programming (DeLP) ---
print("\n--- 4.3 Defeasible Logic Programming (DeLP) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple DeLP...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        import pathlib
        from java.io import StringReader
        from java.util import ArrayList

        # Imports DeLP
        from org.tweetyproject.arg.delp.parser import DelpParser
        from org.tweetyproject.arg.delp.reasoner import DelpReasoner
        from org.tweetyproject.arg.delp.semantics import GeneralizedSpecificity
        from org.tweetyproject.arg.delp.syntax import DefeasibleLogicProgram

        # Imports FOL/Commons necessaires
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature, FolAtom
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate

        print("Imports DeLP, FOL et Commons necessaires reussis.")

        # --- Parsing du programme DeLP ---
        delp_filename = "birds2.txt"
        delp_filepath = pathlib.Path("resources") / delp_filename
        delp_program = None
        parser_delp = DelpParser()

        if not delp_filepath.is_file():
            print(f"ERREUR: Fichier DeLP requis '{delp_filepath}' non trouve !")
            print("   Utilisation d'un exemple integre (birds.txt simplifie).")
            birds_program_str = """
            Bird(X) <- Chicken(X). Bird(X) <- Penguin(X). ~Fly(X) <- Penguin(X).
            Chicken(tina). Penguin(tweety). Scared(tina).
            Fly(X) -< Bird(X). ~Fly(X) -< Chicken(X). Fly(X) -< Chicken(X), Scared(X).
            """
            try:
                 string_reader = StringReader(birds_program_str)
                 delp_program = parser_delp.parseBeliefBase(string_reader); string_reader.close()
                 print("Programme charge avec succes depuis la chaine integree.")
            except Exception as e_str: print(f"Erreur parsing chaine integree: {e_str}"); delp_program = None
        else:
            print(f"\nChargement du programme DeLP depuis: {delp_filepath}")
            try:
                 delp_program = parser_delp.parseBeliefBaseFromFile(str(delp_filepath))
                 print("Programme charge avec succes depuis le fichier.")
            except Exception as e_file: print(f"Erreur chargement fichier: {e_file}"); delp_program = None


        # --- Raisonnement DeLP ---
        if delp_program is not None:
            print("\nProgramme DeLP charge:\n", str(delp_program))

            # DEBUG Signature
            sig_delp = None
            try:
                sig_delp = delp_program.getSignature()
                print("\nSignature extraite par DelpParser:")
                print(sig_delp)
                if hasattr(sig_delp, 'getPredicates'): print("Predicates:", sig_delp.getPredicates())
                if hasattr(sig_delp, 'getConstants'): print("Constants:", sig_delp.getConstants())
            except Exception as e_sig: print(f"Erreur recuperation/affichage signature: {e_sig}")

            # Construction programmatique des requetes
            # IMPORTANT: Utiliser les predicats et constantes du programme charge!
            # Le fichier birds2.txt utilise: Fly (pas Flies), Bird, Penguin, Wings
            # Constantes: tweety, opus
            print("\nConstruction programmatique des requetes FOL...")
            queries_fol_obj = {}
            try:
                # Predicats et constantes correspondant au programme birds2.txt
                Fly = Predicate("Fly", 1)  # PAS "Flies" !
                Bird = Predicate("Bird", 1)
                tweety = Constant("tweety")
                opus = Constant("opus")

                # Construire les FolAtom
                args_tweety = ArrayList([tweety])
                args_opus = ArrayList([opus])

                # Requetes sur les predicats qui existent dans le programme
                queries_fol_obj["Fly(tweety)"] = FolAtom(Fly, JObject(args_tweety, "java.util.List"))
                queries_fol_obj["Fly(opus)"] = FolAtom(Fly, JObject(args_opus, "java.util.List"))
                queries_fol_obj["Bird(tweety)"] = FolAtom(Bird, JObject(args_tweety, "java.util.List"))
                queries_fol_obj["Bird(opus)"] = FolAtom(Bird, JObject(args_opus, "java.util.List"))

                print("Requetes construites programmatiquement.")

            except Exception as e_build:
                print(f"Erreur lors de la construction programmatique des requetes: {e_build}")
                queries_fol_obj = None

            if queries_fol_obj:
                reasoner_delp = DelpReasoner(GeneralizedSpecificity())
                FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")

                print("\nEvaluation des requetes DeLP:")
                print("(YES=justifie, NO=refute, UNDECIDED=indetermine)")
                for q_str, query_formula_obj in queries_fol_obj.items():
                    print(f"  Querying '{q_str}'...", end="")
                    try:
                        result_delp = reasoner_delp.query(delp_program, JObject(query_formula_obj, FolFormula_class))
                        print(f" Resultat: {result_delp}")
                    except jpype.JException as e_query_java:
                        print(f" ERREUR JAVA: {e_query_java.message()}")
                    except Exception as e_query_py:
                         print(f" ERREUR PYTHON: {e_query_py}")
                
                # Explication des resultats attendus pour birds2.txt:
                # - tweety est un Penguin donc Bird(tweety)=YES mais Fly(tweety)=NO (les pingouins ne volent pas)
                # - opus est un Bird mais pas de regle defaisable pour Fly(opus) -> UNDECIDED ou NO
                print("\nNote: Fly(tweety)=NO car ~Fly(X) -< Penguin(X) l'emporte sur Fly(X) -< Bird(X)")
            else:
                 print("\nConstruction des requetes echouee, raisonnement saute.")
        else:
            print("\nAucun programme DeLP charge ou erreur de chargement, raisonnement saute.")

    except ImportError as e: print(f"Erreur d'import pour DeLP/Commons/FOL : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 4.4 Assumption-Based Argumentation (ABA)
<a id="4.4"></a>

*(Section à compléter avec l'exemple `AbaExample.java`)*

En ABA, certains littéraux sont désignés comme des **hypothèses** (assumptions). Les arguments sont dérivés en utilisant des règles logiques (similaires à ASPIC+) à partir de ces hypothèses. Une attaque d'un argument vers un autre se produit si la conclusion du premier est le **contraire** d'une hypothèse utilisée dans le second.

* **`AbaTheory`**: Contient les règles, l'ensemble des hypothèses, et la définition des contraires.
* **Logique sous-jacente**: Peut être PL ou FOL.
* **Raisonnement**: Souvent basé sur la conversion en AF de Dung (`FlatAbaReasoner`, `PreferredReasoner`).

In [ ]:
# --- 4.4 Assumption-Based Argumentation (ABA) ---
print("\n--- 4.4 Assumption-Based Argumentation (ABA) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ABA...")
    try:
        # Imports (inchangés)
        import jpype; from jpype.types import *; import pathlib; from java.io import StringReader
        from org.tweetyproject.arg.aba.parser import AbaParser
        from org.tweetyproject.arg.aba.syntax import AbaTheory, Assumption
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import Proposition, PlFormula
        from org.tweetyproject.logics.fol.parser import FolParser
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature
        from org.tweetyproject.logics.pl.sat import SatSolver, Sat4jSolver
        from org.tweetyproject.arg.aba.reasoner import FlatAbaReasoner, PreferredReasoner
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.arg.dung.syntax import DungTheory

        print("✔️ Imports ABA et dépendances réussis.")

        # --- Exemple 1: Propositional Logic (PL) ---
        print("\n--- Exemple ABA avec Logique Propositionnelle ---")
        # (Code PL inchangé, il fonctionnait)
        SatSolver.setDefaultSolver(Sat4jSolver())
        pl_parser_for_aba = PlParser()
        aba_parser_pl = AbaParser(pl_parser_for_aba)
        aba_pl_filename = "example2.aba"
        aba_pl_filepath = pathlib.Path("resources") / aba_pl_filename
        aba_theory_pl = None
        if not aba_pl_filepath.is_file():
            print(f"❌ ERREUR: Fichier ABA (PL) requis '{aba_pl_filepath}' non trouvé !")
        else:
            print(f"Chargement ABA (PL) depuis fichier: {aba_pl_filepath}")
            try:
                 aba_theory_pl = aba_parser_pl.parseBeliefBaseFromFile(str(aba_pl_filepath))
                 print("✔️ Théorie ABA (PL) chargée depuis fichier.")
            except Exception as e_fpl: print(f"❌ Erreur chargement fichier ABA (PL): {e_fpl}")

        if aba_theory_pl is not None:
            print("\nThéorie ABA (PL):", str(aba_theory_pl))
            reasoner_flat_pref = FlatAbaReasoner(Semantics.PREFERRED_SEMANTICS)
            reasoner_pref_aba = PreferredReasoner()
            assumption_pl = Assumption(Proposition("a"))
            print("\nRequêtes sur la théorie ABA (PL):")
            print(f" - Query '{assumption_pl}' (Flat Preferred)? {reasoner_flat_pref.query(aba_theory_pl, assumption_pl)}")
            print(f" - Query '{assumption_pl}' (ABA Preferred)? {reasoner_pref_aba.query(aba_theory_pl, assumption_pl)}")


        # --- Exemple 2: First-Order Logic (FOL) ---
        print("\n\n--- Exemple ABA avec Logique du Premier Ordre ---")
        aba_fol_filename = "smp_fol.aba"
        aba_fol_filepath = pathlib.Path("resources") / aba_fol_filename
        aba_theory_fol = None

        if not aba_fol_filepath.is_file():
             print(f"❌ ERREUR: Fichier ABA (FOL) requis '{aba_fol_filepath}' non trouvé !")
        else:
            fol_parser_for_aba = FolParser()
            # ** CORRECTION : Ajouter des sauts de ligne '\n' **
            sig_fol_str = """
            Male = {a, b}
            Female = {c, d}
            type(Pair(Male, Female))
            type(ContraryPair(Male, Female))
            type(MPrefers(Male, Female, Female))
            type(WPrefers(Female, Male, Male))
            """
            try:
                 sig_fol_aba = fol_parser_for_aba.parseSignature(sig_fol_str)
                 fol_parser_for_aba.setSignature(sig_fol_aba)
                 print("✔️ Signature FOL pour ABA définie.")

                 aba_parser_fol = AbaParser(fol_parser_for_aba)
                 # Important si les termes sont séparés par ';' dans le .aba
                 # smp_fol.aba utilise bien ';', donc on décommente :
                 aba_parser_fol.setSymbolComma(";")
                 print("ℹ️ Utilisation de ';' comme séparateur pour le parser ABA FOL.")


                 print(f"Chargement ABA (FOL) depuis fichier: {aba_fol_filepath}")
                 try:
                      aba_theory_fol = aba_parser_fol.parseBeliefBaseFromFile(str(aba_fol_filepath))
                      print("✔️ Théorie ABA (FOL) chargée depuis fichier.")
                 except Exception as e_ffol: print(f"❌ Erreur chargement fichier ABA (FOL): {e_ffol}")

                 if aba_theory_fol is not None:
                       print("\nThéorie ABA (FOL):\n", str(aba_theory_fol))
                       # Raisonnement ABA (FOL)
                       reasoner_flat_stable_fol = FlatAbaReasoner(Semantics.STABLE_SEMANTICS)
                       reasoner_pref_aba_fol = PreferredReasoner()
                       FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
                       # Vérifier que 'Pair(a,d)' est bien une assumption dans smp_fol.aba
                       # Elle l'est, comme 'Pair(b,c)', etc.
                       assumption_fol_query_str = "Pair(a,d)"
                       assumption_fol = Assumption(JObject(fol_parser_for_aba.parseFormula(assumption_fol_query_str), FolFormula_class))
                       print("\nRequêtes sur la théorie ABA (FOL):")
                       # print(f" - Modèles (Flat Stable): {reasoner_flat_stable_fol.getModels(aba_theory_fol)}")
                       print(f" - Query '{assumption_fol}' (ABA Preferred)? {reasoner_pref_aba_fol.query(aba_theory_fol, assumption_fol)}")

            except jpype.JException as e_sig_java: print(f"❌ Erreur Java lors de la config FOL pour ABA: {e_sig_java.message()}")
            except Exception as e_sig_py: print(f"❌ Erreur Python lors de la config FOL pour ABA: {e_sig_py}")

    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour ABA : {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 4.5 Argumentation Déductive (PL)
<a id="4.5"></a>

Cette approche, souvent associée à Besnard & Hunter, construit des arguments comme des paires `(Support, Conclusion)` où `Support` est un sous-ensemble minimal et consistant de la base de connaissances qui implique logiquement la `Conclusion`.

* **`DeductiveKnowledgeBase`**: Une simple `PlBeliefSet` dans l'implémentation de Tweety.
* **Argument**: Généré implicitement par le raisonneur.
* **Attaque**: Un argument `(S1, C1)` attaque `(S2, C2)` si `C1` est la négation d'un élément de `S2` (attaque "undercut" classique) ou la négation de `C2` (attaque "rebut" classique).
* **Raisonnement (`SimpleDeductiveReasoner`)**: Construit un arbre d'arguments/contre-arguments pour une requête donnée.
    * **`Categorizer`**: Détermine la force initiale d'un argument (ex: `ClassicalCategorizer`).
    * **`Accumulator`**: Agrège la force des arguments/contre-arguments dans l'arbre (ex: `SimpleAccumulator`).

In [ ]:
# --- 4.5 Argumentation Déductive (PL) ---
# Correction: Le package pour ClassicalCategorizer et SimpleAccumulator était 'categorizer' et 'accumulator', pas 'semantics'.
try:
    from org.tweetyproject.logics.pl.parser import PlParser
    from org.tweetyproject.arg.deductive.syntax import DeductiveKnowledgeBase
    from org.tweetyproject.arg.deductive.reasoner import SimpleDeductiveReasoner, AbstractDeductiveArgumentationReasoner
    # Correction des imports:
    from org.tweetyproject.arg.deductive.categorizer import ClassicalCategorizer
    from org.tweetyproject.arg.deductive.accumulator import SimpleAccumulator

    from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver # Souvent nécessaire en interne

    # --- Initialisation ---
    SatSolver.setDefaultSolver(Sat4jSolver())
    parser_ded = PlParser()
    kb_ded = DeductiveKnowledgeBase() # Est essentiellement un PlBeliefSet

    # Base de connaissances de l'exemple
    formulas_ded = ["s", "!s || h", "f", "!f || !h", "v", "!v || !h"]
    for f_str in formulas_ded: kb_ded.add(parser_ded.parseFormula(f_str))

    print("KB Déductive:\n", kb_ded)

    # --- Raisonnement ---
    # Utilise un catégoriseur classique et un accumulateur simple
    ded_reasoner = SimpleDeductiveReasoner(ClassicalCategorizer(), SimpleAccumulator())

    query_ded = parser_ded.parseFormula("h") # La requête est "h"
    result_ded = ded_reasoner.query(kb_ded, query_ded)

    print(f"\nQuery '{query_ded}'? {result_ded}") # Devrait être ACCEPTED ou REJECTED

except ImportError as e:
     print(f"❌ Erreur d'import pour Argumentation Déductive : {e}")
     print("   Vérifiez le JAR 'org.tweetyproject.arg.deductive'.")
except Exception as e:
     print(f"❌ Erreur lors de l'exécution de l'exemple Déductif: {e}")
     import traceback
     traceback.print_exc()

### 4.6 Answer Set Programming (ASP)
<a id="4.6"></a>

ASP est un paradigme de programmation déclarative puissant pour résoudre des problèmes combinatoires complexes, souvent utilisé en représentation de connaissances et raisonnement. Un programme ASP est un ensemble de règles logiques (similaires à Prolog mais avec une sémantique différente basée sur les "modèles stables" ou "answer sets").

* **Syntaxe :** Règles de la forme `tete :- corps.` où `corps` est une conjonction de littéraux. Utilise la négation par défaut (`not`) et la négation classique (`-`). Permet les contraintes (règles sans tête) et les règles de choix/agrégats.
* **Sémantique :** Les "answer sets" sont des ensembles minimaux d'atomes vrais qui satisfont toutes les règles du programme.
* **Tweety :** Permet de définir des `Program`mes ASP (`org.tweetyproject.lp.asp.syntax`). Le raisonnement nécessite un **solveur externe** comme **Clingo** (de la suite Potassco). Tweety fournit `ClingoSolver` pour l'interfacer, ainsi qu'un `GringoGrounder` (Gringo est le "grounder" de Clingo qui instancie les variables).
* **Pré-requis :** Nécessite l'installation de **Clingo** ([Potassco](https://potassco.org/)) et la configuration du chemin vers l'exécutable Clingo dans `EXTERNAL_TOOLS['CLINGO']` (Cellule 9).

In [ ]:
# --- 4.6 Answer Set Programming (ASP) ---
print("\n--- 4.6 Answer Set Programming (ASP) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ASP...")
    asp_imports_ok = False
    try:
        # Imports (inchangés)
        import jpype
        from jpype.types import *
        from java.util import ArrayList, List as JavaList, Collection
        from org.tweetyproject.lp.asp.syntax import (
            Program, ASPRule, ASPAtom, DefaultNegation, StrictNegation,
            ClassicalHead, AggregateHead, AggregateAtom, ASPHead, ASPBodyElement
        )
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate, Variable
        from org.tweetyproject.lp.asp.reasoner import ClingoSolver
        from org.tweetyproject.lp.asp.grounder import GringoGrounder
        from org.tweetyproject.lp.asp.semantics import AnswerSet

        # Récupérer get_tool_path
        if 'get_tool_path' not in globals() or 'EXTERNAL_TOOLS' not in globals():
            raise NameError("La fonction 'get_tool_path' ou 'EXTERNAL_TOOLS' n'est pas définie.")

        # Récupérer le chemin Clingo configuré par l'utilisateur
        CLINGO_PATH = get_tool_path('CLINGO') # Renvoie None si non configuré ou invalide

        print("✔️ Imports ASP réussis.")
        asp_imports_ok = True

        # --- Programme ASP Simple (Construction inchangée) ---
        print("\n--- Programme ASP Simple ---")
        p = ASPAtom("p"); r_atom = ASPAtom("r"); q = ASPAtom("q"); b_atom = ASPAtom("b") # Renommé r -> r_atom
        body1 = ArrayList(); body1.add(DefaultNegation(r_atom))
        r1_asp = ASPRule(ClassicalHead(p), JObject(body1, JavaList))
        body2 = ArrayList(); body2.add(StrictNegation(q)); body2.add(DefaultNegation(b_atom))
        r2_asp = ASPRule(ClassicalHead(r_atom), JObject(body2, JavaList))
        body3 = ArrayList(); body3.add(b_atom)
        r3_asp = ASPRule(ClassicalHead(StrictNegation(q)), JObject(body3, JavaList))
        body4 = ArrayList()
        r4_asp = ASPRule(ClassicalHead(b_atom), JObject(body4, JavaList))
        rules_prog1 = ArrayList([r1_asp, r2_asp, r3_asp, r4_asp])
        program1 = Program(JObject(rules_prog1, Collection))
        print("Programme 1:\n", str(program1))


        # --- Programme ASP Suspects (Construction inchangée) ---
        print("\n--- Programme ASP Suspects ---")
        motive = Predicate("motive", 1); guilty = Predicate("guilty", 1); innocent = Predicate("innocent", 1)
        harry = Constant("harry"); sally = Constant("sally")
        Suspect = Variable("Suspect")
        motive_h = ASPAtom(motive, [harry]); motive_s = ASPAtom(motive, [sally]) # Mettre les termes dans une liste
        guilty_h = ASPAtom(guilty, [harry])
        innocent_S = ASPAtom(innocent, [Suspect]); motive_S = ASPAtom(motive, [Suspect])
        guilty_S = ASPAtom(guilty, [Suspect])
        r1_sus = ASPRule(ClassicalHead(motive_h), JObject(ArrayList(), JavaList))
        r2_sus = ASPRule(ClassicalHead(motive_s), JObject(ArrayList(), JavaList))
        r3_sus = ASPRule(ClassicalHead(guilty_h), JObject(ArrayList(), JavaList))
        body4_sus = ArrayList(); body4_sus.add(motive_S); body4_sus.add(DefaultNegation(guilty_S))
        r4_sus = ASPRule(ClassicalHead(innocent_S), JObject(body4_sus, JavaList))
        rules_prog2 = ArrayList([r1_sus, r2_sus, r3_sus, r4_sus])
        program2 = Program(JObject(rules_prog2, Collection))
        print("Programme 2:\n", str(program2))


        # --- Raisonnement avec Clingo ---
        if not CLINGO_PATH:
            print("\n⚠️ Clingo non configuré ou chemin invalide dans EXTERNAL_TOOLS['CLINGO'].")
            print("   Veuillez installer Clingo (via pip/conda ou manuellement) et configurer le chemin.")
            print("   Raisonnement ASP sauté.")
        else:
            print(f"\nUtilisation de Clingo trouvé/configuré à: {CLINGO_PATH}")
            try:
                # Instancier AVEC le chemin (doit être un REPERTOIRE, pas l'exe)
                solver_instance = ClingoSolver(JString(CLINGO_PATH))
                print(f"\nCalcul des Answer Sets pour Programme 2...")
                # getModels(Program)
                answer_sets_collection = solver_instance.getModels(program2)

                print(f"Answer Sets trouvés ({answer_sets_collection.size()}):")
                if answer_sets_collection.isEmpty():
                    print("   (Aucun Answer Set)")
                else:
                    count = 1
                    for ans_set in answer_sets_collection:
                        # AnswerSet extends InterpretationSet - utiliser toString() ou itérer directement
                        # L'objet AnswerSet est itérable (Set<ASPLiteral>)
                        try:
                            atoms_in_set = ", ".join(sorted([str(atom) for atom in ans_set]))
                            print(f"   - AS {count}: {{{atoms_in_set}}}")
                        except Exception:
                            # Fallback: utiliser toString()
                            print(f"   - AS {count}: {ans_set}")
                        count += 1

                # --- Grounding (Optionnel) ---
                # Note: Dans les versions modernes de Clingo (5.0+), gringo a été fusionné dans clingo.
                # GringoGrounder de TweetyProject cherche un exécutable 'gringo' séparé qui n'existe plus.
                # Cette section est donc désactivée car non fonctionnelle avec clingo moderne.
                print("\nℹ️ Grounding désactivé (GringoGrounder incompatible avec Clingo 5.0+)")
                print("   Note: Clingo 5.0+ intègre le grounding en interne. L'ancienne API Gringo")
                print("   standalone n'est plus supportée. ClingoSolver fait déjà le grounding.")

                # Code de grounding désactivé:
                # print("\nTentative de Grounding...")
                # try:
                #     grounder_instance = GringoGrounder(JString(CLINGO_PATH))
                #     ground_program = grounder_instance.getGroundProgram(program2)
                #     print("   ✔️ Grounding réussi.")
                #     print(f"     (Nombre de règles groundées: {ground_program.size()})")
                # except Exception as e_ground:
                #     print(f"   ⚠️ Grounding non disponible: {e_ground}")
                #     print("   (Gringo 4.x requis, mais Clingo 5.0+ intègre le grounding)")

            except jpype.JException as e_clingo_java:
                print(f"❌ Erreur Java lors de l'appel à Clingo: {e_clingo_java.message()}")
                print("   Vérifiez que le chemin vers Clingo est correct et que Clingo fonctionne.")
                # print(e_clingo_java.stacktrace())
            except Exception as e_clingo_py:
                print(f"❌ Erreur Python lors de l'appel à Clingo: {e_clingo_py}")
                import traceback; traceback.print_exc()


    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour ASP : {e}. Vérifiez le JAR 'lp.asp'.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale ASP: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue ASP: {e_gen}"); import traceback; traceback.print_exc()

---

## Résumé

Ce notebook a couvert:
- **ASPIC+**: Règles strictes/défaisables, conversion vers Dung AF
- **DeLP**: Programmation logique défaisable
- **ABA**: Argumentation basée sur les hypothèses
- **Argumentation Déductive**: Démonstrations en logique propositionnelle
- **ASP**: Answer Set Programming avec Clingo

## Prochaines étapes

Le notebook suivant explore les extensions avancées: ADF, frameworks pondérés, argumentation sociale et ranking.

---

**Navigation**: [← Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7-Advanced-Argumentation →](Tweety-7-Advanced-Argumentation.ipynb)